In [3]:
import glob
import os

# import pdfplumber
# from tqdm import tqdm
# from sentence_transformers import SentenceTransformer
# import psycopg2
# from psycopg2.extras import execute_values

In [ ]:
# lancer postgresql : sudo service postgresql start
# vérifier le statut de postgresql : sudo service postgresql status
# se connecter au serveur postgresql dans WSL : sudo -u postgres psql
# liste des base de données : SELECT datname FROM pg_database;
# créer une bdd : CREATE DATABASE chatnote TEMPLATE template0;
# se connecter à la bdd : \c chatnote
# créer l'extension pgvector : CREATE EXTENSION vector;
# liste des extensions : \dx


In [ ]:
# Se connecter avec pgAdmin 4 depuis Windows à PostgreSQL dans WSL :

# 1. Assurez-vous que PostgreSQL dans WSL écoute les connexions externes (pas seulement localhost dans WSL) :
# Modifiez postgresql.conf dans WSL (généralement /etc/postgresql/16/main/postgresql.conf): Changez listen_addresses = 'localhost' en listen_addresses = '*'.
# Modifiez pg_hba.conf dans WSL (généralement /etc/postgresql/16/main/pg_hba.conf) pour autoriser les connexions depuis votre IP Windows ou le réseau WSL. Pour commencer, vous pouvez ajouter (soyez prudent avec trust en production) :
# host    all             all             0.0.0.0/0            md5  # Ou scram-sha-256
# # Ou plus spécifiquement pour l'interface réseau de WSL
# host    all             all             <IP_de_votre_adaptateur_WSL> trust # Moins sécurisé, préférez md5 ou scram-sha-256 avec mot de passe
# Trouvez l'IP de votre adaptateur WSL en exécutant ip addr dans WSL (cherchez l'interface eth0 ou similaire).
# Redémarrez le service PostgreSQL dans WSL : sudo service postgresql restart

# 2. Dans pgAdmin 4 (sur Windows) :
# Créez une nouvelle connexion serveur.
# Nom d'hôte/adresse : Utilisez l'adresse IP de votre distribution WSL. Vous pouvez la trouver en exécutant hostname -I dans votre terminal WSL. Si cela ne fonctionne pas directement, localhost peut parfois fonctionner si le port forwarding est bien configuré par WSL, mais l'IP explicite est plus fiable.
# Port : 5432 (sauf si vous l'avez changé).
# Base de données de maintenance : postgres ou le nom de votre base de données.
# Nom d'utilisateur / Mot de passe : Ceux de votre utilisateur PostgreSQL configuré dans WSL.

In [13]:
PATH_NOTES_wsl = r"/mnt/g/Esp-mission-Patrimoine/documentLibrary/Habitat_Amenagement/OBSERVATOIRE/02_ACTIVITES/ETUDES/ETUDES_ADIL/NOTES"
prepro_dir = 'preprocessing'
nom_notes = [note.split('/')[-1].split(".")[0] for note in glob.glob(os.path.join(PATH_NOTES_wsl, prepro_dir, '*')) if 'meta' not in note]
texte_dir = 'texte'

In [14]:
nom_notes

[]

In [ ]:
# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="chatnote",
    user="postgres",
    password="",
    host="localhost",
    port="5432"
)

# Create a table to store embeddings with metadata
with conn.cursor() as cursor:
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS embeddings (
        id SERIAL PRIMARY KEY,
        note_name TEXT,
        page_number INTEGER,
        text TEXT,
        embedding vector(384)  -- Adjust the dimension based on your model
    )
    """)
    conn.commit()

# Assuming PATH_NOTES, chemin_notes, nom_notes, and numero_notes are defined
prepro_dir = 'preprocessing'
os.makedirs(os.path.join(PATH_NOTES, prepro_dir), exist_ok=True)
texte_dir = 'texte'

for chemin, nom, numero in tqdm(zip(chemin_notes, nom_notes, numero_notes), desc="Processing Notes"):
    pdf = pdfplumber.open(chemin)
    note_dir = f"{nom.split('.')[0]}"
    page_dir = os.path.join(PATH_NOTES, prepro_dir, note_dir, texte_dir)
    os.makedirs(page_dir, exist_ok=True)

    for idx, page in tqdm(enumerate(pdf.pages), desc="Processing Pages", leave=False):
        text = page.extract_text()
        text_clean = remove_occurences_3(text)

        # Generate embeddings
        embedding = model.encode(text_clean)

        # Insert embeddings and metadata into the table
        with conn.cursor() as cursor:
            execute_values(
                cursor,
                "INSERT INTO embeddings (document_name, page_number, text, embedding) VALUES %s",
                [(nom, idx + 1, text_clean, embedding.tolist())]
            )
            conn.commit()

        # Optionally, save the cleaned text to a file
        with open(f"{page_dir}/page_{idx+1}.txt", "w", encoding='utf-8') as f:
            f.write(text_clean)

# Close the connection
conn.close()
